In [4]:
import os
import sagemaker
print(sagemaker.__version__)

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel
import time
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/st'

role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

2.112.2


In [5]:
!rm -rf keybert-pytorch.tar.gz
# !cp pretrained_model/mae_pretrain_vit_base.pth model/checkpoint-best.pth
!cd model && tar -czvf ../keybert-pytorch.tar.gz *

!aws s3 cp keybert-pytorch.tar.gz s3://$bucket/keybert-pytorch.tar.gz

code/
code/requirements.txt
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/inference.py
upload: ./keybert-pytorch.tar.gz to s3://sagemaker-us-east-1-726335585155/keybert-pytorch.tar.gz


In [ ]:
# instance_type = 'local'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)
endpoint_name="keybert-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
pytorch_model = PyTorchModel(model_data='s3://{}/keybert-pytorch.tar.gz'.format(bucket), role=role,
                             entry_point='inference.py', framework_version='1.7.1', py_version='py36', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1, endpoint_name=endpoint_name)

-----

In [ ]:
import sagemaker
import json
sm = sagemaker.Session().sagemaker_runtime_client

prompt = "The best part of Amazon SageMaker is that it makes machine learning easy."

response = sm.invoke_endpoint(
    EndpointName=endpoint_name, Body=prompt.encode(encoding="UTF-8"), ContentType="text/csv"
)

result = response["Body"].read().decode()

# result = list(map(float, result[1:-1].split()))
eval(result)

In [ ]:
predictor.delete_endpoint()